In [ ]:
import numpy as np
from multiprocessing import Pool
from functools import reduce
import editdistance
import matplotlib.pyplot as plt

In [ ]:
data = np.load("assemblydata.npy", allow_pickle=True)

In [ ]:
results = data[0]
ref = data[1]
len(results)

In [ ]:
# https://pythonforbiologists.com/a-terrible-genome-assembler-in-six-lines
def cm(d,ds):
    mm = [m(d,e) for e in ds if e != ds]
    return max(mm) if len(mm) > 0 else 0

def m(d,e):
    return max([(s(d,e,o),o,e,d) for o in range(1-len(e),len(d))])

def s(d,e,o):
    return sum([1 for p in range(max(0-o,0), min([len(e)-o, len(e), len(d)-o])) if e[p] == d[p+o]])

def con(x,o,s,c):
    return c[0:max(0,o)] + s +  c[len(s)+o:]

def a(s, o):
    return con(*cm(s, o)) if len(o) == 1 else a(con(*cm(s, o)), [ y for y in o if y != cm(s, o)[2]])

def ah(d):
    return a(d[0],d[1:])

In [ ]:
len(results)

In [ ]:
difs = []
for i, k in enumerate(results[1:]):
    difs.append(editdistance.eval(k, results[i-1]))

In [ ]:
plt.hist(difs, density=True)

In [ ]:
def filterabnormal(a):
    a,b,c = a
    e1 = editdistance.eval(a,b)
    e2 = editdistance.eval(b,c)
    return e1+e2 < 20


filtered = list(map(lambda x:x[1], filter(filterabnormal, zip(results, results[1:], results[2:]))))

while True:
    prevlen = len(filtered)
    filtered = list(map(lambda x:x[1], filter(filterabnormal, zip(filtered, filtered[1:], filtered[2:]))))
    if prevlen - len(filtered) < 10 :
        break


len(filtered)

In [ ]:
difs2 = []
for i, k in enumerate(filtered[1:]):
    difs2.append(editdistance.eval(k, filtered[i-1]))
plt.hist(difs2)
print(max(difs2))

In [ ]:
filtered[:10]

In [ ]:
ab = filtered[0]


for j in filtered:
    ab = ah([ab, j])

In [ ]:
len(ab)

In [ ]:
ab